In [231]:
import json
import psycopg2
from psycopg2.extras import RealDictCursor

### Get Connection Info

You'll need to create a file named connection_info.json in the same directory as this notebook. There should be a template for this file at *./connection_info\_template.json*.<br/>

Your connection_info.json file should look something like this (with the same keys):<br/>
{'db_host': 'databaseURL', 'db_name': 'databasename', 'db_user': 'databaseusername', 'db_pass': 'usernamepassword'}<br/>

This keeps our database credentials safe; we can share this repo publicly because connection_info.json is in our .gitignore.

In [232]:
with open('./connection_info.json', 'r') as myfile:
    connection_info = json.loads(myfile.read())

In [234]:
db_host = connection_info['db_host']
db_name = connection_info['db_name']
db_user = connection_info['db_user']
db_pass = connection_info['db_pass']

### Connect

Uses the connection info we retrieved from the connection_info.json file. If you're running this code in an AWS Lambda, simply declare db_name, db_user, etc as global variables at the top of the lambda handler file (or as environment variables: https://docs.aws.amazon.com/lambda/latest/dg/configuration-envvars.html).

In [236]:
def create_conn():
    """Connect to a Postgres database with psycopg2.

    Must provide host address, database name, username, and password, as
    global variables: db_name, db_user, db_pass, db_host.
    Connects to port 5432 by default.
    :return: A connection object (used by psycopg2 to query database).
    """
    conn = None
    try:
        conn = psycopg2.connect("dbname={} user={} host={} password={}" \
          .format(db_name,db_user,db_host,db_pass))
    except:
        print("Cannot connect to PostgreSQL database at port 5432. Check \
        database name, Postgresql username and password, and database host \
        address.")
    return conn

### Query

In [237]:
def fetch(conn, query):
    """Execute a Postgres query with psycopg2 and return the results.
    
    Return results as a list of json dictionaries. Row order is preserved.
    Each row is a separate dictionary. Field names are keys. Values can be 
    strings or integers. For example:
    [
     { "First Name": "Bobby", "Age": 30 },
     { "First Name": "Jacques", "Age": 34 }
    ]
    :return: A list of dictionaries.
    """
    # You can delete the cursor_factory argument to return a list of lists:
    cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    cur.mogrify(query)  # exact str that will be sent to db (for debugging)
    cur.execute(query)
    
    '''
    After an execute call, the results can be retrieved from the cursor. The
    cursor object is iterable ("for row in cursor").
    Some helpful methods:
      - fetchone()
      - fetchmany(int)
      - fetchall()
      - rowcount() (read-only, returns number of rows)
      - query() (read-only, returns body of last query)
    '''
    results = cur.fetchall()
    results_as_json = json.dumps(results, default=str, indent=2)
    return results_as_json

### Test it:

In [238]:
fields = '*'
table_name = 'current_electedmember_denormalized'
SQL = f'SELECT {fields} FROM {table_name} LIMIT 2;'

conn = create_conn()
res = fetch(conn, SQL)
print(res)

[
  {
    "politician_id": 24,
    "name_family": "Bennett",
    "name_given": "Carolyn",
    "twitter": "Carolyn_Bennett",
    "riding_name": "Toronto--St. Paul's",
    "riding_province": "ON",
    "party_name_en": "Liberal",
    "start_date": "2015-10-19",
    "headshot": "BennettCarolyn"
  },
  {
    "politician_id": 612,
    "name_family": "McLeod",
    "name_given": "Cathy",
    "twitter": "Cathy_McLeod",
    "riding_name": "Kamloops--Thompson--Cariboo",
    "riding_province": "BC",
    "party_name_en": "Conservative",
    "start_date": "2008-11-18",
    "headshot": "McLeodCathy"
  }
]
